In [93]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy.stats as stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import cross_validation
from sklearn import preprocessing
path='D:/Project Data/Happy_Customer_Bank/'
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline
import math

In [94]:
train=pd.read_csv(path+'Train.csv',encoding='iso-8859-1')
testdata=pd.read_csv(path+'Test.csv',encoding='iso-8859-1')

train['age']=[(2015-(1900+int(i[-2:]))) for i in train['DOB'].values]  

train['Loan_Amount_Applied']=train['Loan_Amount_Applied'].fillna(train['Loan_Amount_Applied'].mean())

train['Loan_Tenure_Applied']=train['Loan_Tenure_Applied'].fillna(train['Loan_Tenure_Applied'].median())

train['Existing_EMI']=train['Existing_EMI'].fillna(0)

train['Loan_Amount_Submitted']=train['Loan_Amount_Submitted'].fillna(train['Loan_Amount_Submitted'].mean())

train['Loan_Tenure_Submitted']=train['Loan_Tenure_Submitted'].fillna(train['Loan_Tenure_Submitted'].median())

train['Interest_Rate']=train['Interest_Rate'].fillna(train['Interest_Rate'].median())

train['Processing_Fee']=train['Processing_Fee'].fillna(train['Processing_Fee'].mean())

train['EMI_Loan_Submitted']=train['EMI_Loan_Submitted'].fillna(train['EMI_Loan_Submitted'].mean())

Loan_Amount_Applied_mean=train['Loan_Amount_Applied'].mean()

Loan_Tenure_Applied_median=train['Loan_Tenure_Applied'].median()

Loan_Amount_Submitted_mean=train['Loan_Amount_Submitted'].mean()

Loan_Tenure_Submitted_median=train['Loan_Tenure_Submitted'].median()

Interest_Rate_median=train['Interest_Rate'].median()

Processing_Fee_mean=train['Processing_Fee'].mean()

EMI_Loan_Submitted_mean=train['EMI_Loan_Submitted'].mean()

In [95]:
print(train.shape)
train=train[(train['age'] < 60) | (train['Disbursed'] >=1)]
train=train[(train['Interest_Rate'] < 30) | (train['Disbursed'] >=1) ]
train=train[(train['Loan_Tenure_Applied'] > 1) | (train['Disbursed'] >=1) ]
train=train[(train['Monthly_Income'] < 600000) | (train['Monthly_Income'] > 6000)]
train=train[(train['Existing_EMI'] <= 0) | (train['Existing_EMI'] >= 1000) | (train['Disbursed'] >=1) ]
print(train.shape)
np.unique(train['Disbursed'],return_counts=True)

(87020, 27)
(47065, 27)


(array([0, 1], dtype=int64), array([45792,  1273], dtype=int64))

In [96]:
Numerical=['age','Monthly_Income','Loan_Amount_Applied', 'Loan_Tenure_Applied','Existing_EMI','Var5', 'Loan_Amount_Submitted', 'Loan_Tenure_Submitted', 'Interest_Rate', 
           'Processing_Fee', 'EMI_Loan_Submitted', ]
Categorical=['Gender', 'City','Mobile_Verified','Var1','Filled_Form', 'Device_Type', 'Var2', 'Source']

Target=['Disbursed']
Target_log=['LoggedIn']


In [97]:
Variables=Numerical+Categorical
Variable_w_log= Variables + Target_log

In [98]:

for var in Categorical:
    lb=LabelEncoder()
    full_data=pd.concat((train[var],testdata[var]),axis=0).astype('str')
    lb.fit(full_data)
    train[var]=lb.transform(train[var].astype('str'))
    testdata[var]=lb.transform(testdata[var].astype('str'))


In [99]:
train.corr()['Disbursed'].order(ascending=False)

Disbursed                1.000000
LoggedIn                 0.749983
Var5                     0.112775
Filled_Form              0.084977
Var4                     0.081043
Mobile_Verified          0.076333
Gender                   0.076329
EMI_Loan_Submitted       0.072815
Loan_Amount_Submitted    0.069310
Processing_Fee           0.051792
age                      0.041576
Var2                     0.014222
Existing_EMI             0.011357
Monthly_Income           0.005193
Loan_Tenure_Submitted    0.003686
City                    -0.003173
Loan_Amount_Applied     -0.016682
Source                  -0.029441
Var1                    -0.087065
Interest_Rate           -0.089128
Loan_Tenure_Applied     -0.133451
Device_Type             -0.416022
Name: Disbursed, dtype: float64

In [100]:
traindata=train[Variables].as_matrix()

In [101]:
traindata.shape

(47065, 19)

In [102]:
from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(train[Target_log], 2, test_size=.5, random_state=42)
for train_index, test_index in sss:
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_s, X_test_s = traindata[train_index], traindata[test_index]
    y_train_s, y_test_s = train[Target_log].values[train_index], train[Target_log].values[test_index]

X_train_s.shape, X_test_s.shape, y_train_s.shape, y_test_s.shape

y_test_s=[i[0] for i in y_test_s]

In [103]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(X_train_s,y_train_s)
predict_class=rfc.predict(X_test_s)
print(accuracy_score(predict_class, y_test_s))
confusion_matrix(predict_class, y_test_s)
#metrics.auc(predict_class, target_test_1)

0.961968299834


C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from IPython.kernel.zmq import kernelapp as app


array([[22444,   861],
       [   34,   194]])

In [104]:
logistic = linear_model.LogisticRegression()
logistic.fit(X_train_s,y_train_s)
predict_class=logistic.predict(X_test_s)
print(accuracy_score(predict_class, y_test_s))
confusion_matrix(predict_class, y_test_s)

0.955254323716


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[22478,  1053],
       [    0,     2]])

In [105]:
bag=ensemble.BaggingClassifier(rfc)
bag.fit(X_train_s,y_train_s)
predict_class2=bag.predict(X_test_s)
print(accuracy_score(predict_class2, y_test_s))
confusion_matrix(predict_class2, y_test_s)


0.962945650788


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[22471,   865],
       [    7,   190]])

In [106]:
train_log=train[Variable_w_log].as_matrix()
train_log.shape

(47065, 20)

In [107]:
sss1 = StratifiedShuffleSplit(train[Target], 2, test_size=.5, random_state=42)
for train_index1, test_index1 in sss1:
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_log, X_test_log = train_log[train_index1], train_log[test_index1]
    y_train_log, y_test_log = train[Target].values[train_index1], train[Target].values[test_index1]

In [108]:
rfc_ = ensemble.RandomForestClassifier()
rfc_.fit(X_train_log,y_train_log)
predict_log=rfc_.predict(X_test_log)
print(accuracy_score(predict_log, y_test_log))
confusion_matrix(predict_log, y_test_log)

0.983682488421


C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from IPython.kernel.zmq import kernelapp as app


array([[22749,   237],
       [  147,   400]])

In [109]:
logistic1 = linear_model.LogisticRegression()
logistic1.fit(X_train_log,y_train_log)
predict_log=logistic1.predict(X_test_log)
print(accuracy_score(predict_log, y_test_log))
confusion_matrix(predict_log, y_test_log)

0.973908978881


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[22876,   594],
       [   20,    43]])

In [110]:
bag2=ensemble.BaggingClassifier(rfc_)
bag2.fit(X_train_log,y_train_log)
predict_log_=bag2.predict(X_test_log)
print(accuracy_score(predict_log_, y_test_log))
confusion_matrix(predict_log_, y_test_log)

0.985297242171


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[22747,   197],
       [  149,   440]])

In [111]:
testdata['age']=[(2015-(1900+int(i[-2:]))) for i in testdata['DOB'].values]  

testdata['Loan_Amount_Applied']=testdata['Loan_Amount_Applied'].fillna(Loan_Amount_Applied_mean)

testdata['Loan_Tenure_Applied']=testdata['Loan_Tenure_Applied'].fillna(Loan_Tenure_Applied_median)

testdata['Existing_EMI']=testdata['Existing_EMI'].fillna(0)

testdata['Loan_Amount_Submitted']=testdata['Loan_Amount_Submitted'].fillna(Loan_Amount_Submitted_mean)

testdata['Loan_Tenure_Submitted']=testdata['Loan_Tenure_Submitted'].fillna(Loan_Tenure_Submitted_median)

testdata['Interest_Rate']=testdata['Interest_Rate'].fillna(Interest_Rate_median)

testdata['Processing_Fee']=testdata['Processing_Fee'].fillna(Processing_Fee_mean)

testdata['EMI_Loan_Submitted']=testdata['EMI_Loan_Submitted'].fillna(EMI_Loan_Submitted_mean)


In [112]:
testdata_final=testdata[Variables].as_matrix()

In [113]:
predict_class_test=logistic.predict(testdata_final)
np.unique(predict_class_test, return_counts=True)
predict_class_test=[[i] for i in predict_class_test]


predict_ense_test=rfc.predict(testdata_final)
np.unique(predict_ense_test, return_counts=True)
predict_ense_test=[[i] for i in predict_ense_test]

predict_bag_test=bag.predict(testdata_final)
np.unique(predict_bag_test, return_counts=True)
predict_bag_test=[[i] for i in predict_bag_test]


In [114]:
testdata_full_final=np.hstack((testdata_final,predict_class_test))
testdata_full_final.shape

predict_log_test=logistic1.predict(testdata_full_final)
predict_log_test=predict_log_test.tolist()
print(np.unique(predict_log_test,return_counts=True))
sub=pd.DataFrame(data={'ID':testdata['ID'],'Disbursed':predict_log_test},columns=['ID','Disbursed'])
#sub.to_csv(path+'sub13.csv',index=False)

(array([0, 1]), array([36994,   723], dtype=int64))


In [115]:
testdata_full_final=np.hstack((testdata_final,predict_ense_test))
testdata_full_final.shape

predict_log_test_=rfc_.predict(testdata_full_final)
predict_log_test_=predict_log_test_.tolist()
print(np.unique(predict_log_test_,return_counts=True))
sub1=pd.DataFrame(data={'ID':testdata['ID'],'Disbursed':predict_log_test_},columns=['ID','Disbursed'])
#sub1.to_csv(path+'ense_sub13.csv',index=False)

(array([0, 1]), array([21119, 16598], dtype=int64))


In [116]:
testdata_full_final=np.hstack((testdata_final,predict_bag_test))
testdata_full_final.shape

predict_log_test_1=bag2.predict(testdata_full_final)
predict_log_test_1=predict_log_test_1.tolist()
print(np.unique(predict_log_test_1,return_counts=True))
sub2=pd.DataFrame(data={'ID':testdata['ID'],'Disbursed':predict_log_test_1},columns=['ID','Disbursed'])
#sub2.to_csv(path+'bag_sub13.csv',index=False)

(array([0, 1]), array([21126, 16591], dtype=int64))
